In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Start a Spark session with the SQLite JDBC driver
spark = SparkSession.builder \
    .appName("SmartSales") \
    .config("spark.jars", "/Users/stephneyaganaga/Desktop/school-assessment/smart-sales-blessing/lib/sqlite-jdbc-3.49.1.0.jar") \
    .getOrCreate()


# Load data from SQLite
df_sales = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlite:/Users/stephneyaganaga/Desktop/school-assessment/smart-sales-blessing/data/dw/smart_sales.db") \
    .option("dbtable", "sale") \
    .option("driver", "org.sqlite.JDBC") \
    .load()


df_sales.show()

df_customer = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlite:/Users/stephneyaganaga/Desktop/school-assessment/smart-sales-blessing/data/dw/smart_sales.db") \
    .option("dbtable", "customer") \
    .option("driver", "org.sqlite.JDBC") \
    .load()

df_products = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlite:/Users/stephneyaganaga/Desktop/school-assessment/smart-sales-blessing/data/dw/smart_sales.db") \
    .option("dbtable", "product") \
    .option("driver", "org.sqlite.JDBC") \
    .load()

df_products.show()

# Register DataFrames as temporary views (if not already done)
df_sales.createOrReplaceTempView("sale")
df_products.createOrReplaceTempView("product")
df_customer.createOrReplaceTempView("customer")

df_sales_with_category_and_customer = spark.sql("""
SELECT s.*, p.category, c.region
FROM sale s
JOIN customer c ON s.customer_id = c.customer_id
JOIN product p ON s.product_id = p.product_id
""")

# Write query using Spark SQL
df_top_customers = spark.sql("""
SELECT c.name, SUM(s.sale_amount) AS total_spent
FROM sale s
JOIN customer c ON s.customer_id = c.customer_id
GROUP BY c.name
ORDER BY total_spent DESC
""")

# Show Spark results
df_top_customers.show()

# Convert to Pandas for use with charts
import pandas as pd
df_top_customers_pd = df_top_customers.toPandas()

df_filtered = df_sales.filter(df_sales.sale_date >= "2023-01-01")

df_grouped = df_sales_with_category_and_customer.groupby("category", "region").sum("sale_amount").show()


df_sales_with_time = df_sales.withColumn(
    "sale_date_parsed", F.to_date(F.col("sale_date"), "M/d/yyyy")
)

df_sales_with_time = df_sales_with_time.withColumn(
    "year", F.year(F.col("sale_date_parsed"))
).withColumn(
    "month", F.month(F.col("sale_date_parsed"))
).withColumn(
    "quarter", F.quarter(F.col("sale_date_parsed"))
)

df_sales_with_time.groupby("year", "quarter", "month").sum("sale_amount").show()



import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(data=df_top_customers_pd, x="name", y="total_spent")
plt.xticks(rotation=45)
plt.show()

25/04/19 11:43:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/19 11:43:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+-------+-----------+----------+-----------+---------+--------+-----------+--------------------+--------------+
|sale_id|customer_id|product_id|sale_amount|sale_date|store_id|campaign_id|    discount_percent|         state|
+-------+-----------+----------+-----------+---------+--------+-----------+--------------------+--------------+
|    550|       1008|       102|       39.1| 1/6/2024|     404|          0|2.430000000000000000|         Texas|
|    551|       1009|       105|      19.78| 1/6/2024|     403|          0|2.750000000000000000|       Florida|
|    552|       1004|       107|      335.1|1/16/2024|     404|          0|0.850000000000000000|    California|
|    553|       1006|       102|      195.5|1/16/2024|     406|          0|1.710000000000000000|      New York|
|    554|       1005|       102|      117.3|1/25/2024|     405|          0|1.850000000000000000|      Illinois|
|    555|       1001|       101|    2379.36|1/25/2024|     401|          0|0.610000000000000000|  Pennsy

NameError: name 'result' is not defined